In [10]:
from huggingface_hub import notebook_login
notebook_login()

In [2]:
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM, set_seed, BitsAndBytesConfig

set_seed(1234)

# Identify the checkpoint to use
model_checkpoint = "beomi/gemma-ko-7b"

# Load the tokenizer
tokenizer = AutoTokenizer.from_pretrained(model_checkpoint)

# Define the BitsAndBytesConfig for 8-bit quantization
bnb_config = BitsAndBytesConfig(
    load_in_8bit=True,
    bnb_8bit_compute_dtype=torch.float16,
    bnb_8bit_use_double_quant=False
)

# Load the model with 8-bit quantization and move it to GPU
model = AutoModelForCausalLM.from_pretrained(
    model_checkpoint,
    quantization_config=bnb_config,
    device_map="cuda"
)

Unused kwargs: ['bnb_8bit_compute_dtype', 'bnb_8bit_use_double_quant']. These kwargs are not used in <class 'transformers.utils.quantization_config.BitsAndBytesConfig'>.
`config.hidden_act` is ignored, you should use `config.hidden_activation` instead.
Gemma's activation function will be set to `gelu_pytorch_tanh`. Please, use
`config.hidden_activation` if you want to override this behaviour.
See https://github.com/huggingface/transformers/pull/29402 for more details.


Loading checkpoint shards:   0%|          | 0/6 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/132 [00:00<?, ?B/s]

안녕하세요, 저는 인공지능 모델입니다. 저는 인공지능 모델이기 때문에 제가 하는 말은 모두 사실입니다. 저는 인공지능 모델이기 때문에 제가 하는 말은 모두


In [3]:
# print out the model layers
print(model)
print(model.dtype)

GemmaForCausalLM(
  (model): GemmaModel(
    (embed_tokens): Embedding(256000, 3072, padding_idx=0)
    (layers): ModuleList(
      (0-27): 28 x GemmaDecoderLayer(
        (self_attn): GemmaSdpaAttention(
          (q_proj): Linear8bitLt(in_features=3072, out_features=4096, bias=False)
          (k_proj): Linear8bitLt(in_features=3072, out_features=4096, bias=False)
          (v_proj): Linear8bitLt(in_features=3072, out_features=4096, bias=False)
          (o_proj): Linear8bitLt(in_features=4096, out_features=3072, bias=False)
          (rotary_emb): GemmaRotaryEmbedding()
        )
        (mlp): GemmaMLP(
          (gate_proj): Linear8bitLt(in_features=3072, out_features=24576, bias=False)
          (up_proj): Linear8bitLt(in_features=3072, out_features=24576, bias=False)
          (down_proj): Linear8bitLt(in_features=24576, out_features=3072, bias=False)
          (act_fn): PytorchGELUTanh()
        )
        (input_layernorm): GemmaRMSNorm((3072,), eps=1e-06)
        (post_attenti

In [25]:
# Example text for the model to process
text = "파이썬으로 bfs 짜줘"
inputs = tokenizer(text, return_tensors="pt").to("cuda")

# Generate output
with torch.no_grad():
    outputs = model.generate(inputs["input_ids"], max_length=50)

# Decode the output to text
decoded_output = tokenizer.decode(outputs[0], skip_special_tokens=True)
print(decoded_output)

파이썬 코드 아무거나 짜줘요. 100줄 정도로요. 100줄이면 100줄이라고 말해주세요. 100줄이면 


In [24]:
# create a list of turns for our conversation, each conversation has to start
# with a user turn and must alternate between user/model
chat = [
    {"role": "user", "content": "Write a hello world program in Python please."}
]
# next we'll use the apply_chat_template from our tokenizer, this will convert
# our list into the conversation prompt and at the same time
# convert the prompt to tokens all in one call.
token_inputs = tokenizer.apply_chat_template(chat, tokenize=True, return_tensors="pt", add_generation_prompt=True).to("cuda")

# an important thing to note about the apply_chat_template, it returns the tensor of the input ids directly.
# unlike how we called the tokenizer before and got back a dict with the input_ids entry,
# it's not a huge change, just need to keep in mind this returns a slightly different structure.

# call inference on the model
token_outputs = model.generate(input_ids=token_inputs, do_sample=True, max_new_tokens=500, temperature=.5)
new_tokens = token_outputs[0][token_inputs.shape[-1]:]
decoded_output = tokenizer.decode(new_tokens, skip_special_tokens=True)
print(decoded_output)


<|access_token|>: {access_token}
<|refresh_token|>: {refresh_token}
<|expires_in|>: {expires_in}
<|last_access_date|>: {last_access_date}
<|photo_count|>: {photo_count}
<|photo_size|>: {photo_size}
<|photo_type|>: {photo_type}
<|photo_thumb_size|>: {photo_thumb_size}
<|photo_title|>: {photo_title}
<|photo_description|>: {photo_description}
<|photo_date|>: {photo_date}
<|photo_thumbnail_url|>: {photo_thumbnail_url}
<|photo_original_url|>: {photo_original_url}
<|photo_original_size|>: {photo_original_size}
<|photo_cropped_size|>: {photo_cropped_size}
<|photo_cropped_url|>: {photo_cropped_url}
<|photo_cropped_original_size|>: {photo_cropped_original_size}
<|photo_cropped_original_url|>: {photo_cropped_original_url}
<|photo_cropped_thumbnail_size|>: {photo_cropped_thumbnail_size}
<|photo_cropped_thumbnail_url|>: {photo_cropped_thumbnail_url}
<|photo_cropped_original_thumbnail_size|>: {photo_cropped_original_thumbnail_size}
<|photo_cropped_original_thumbnail_url|>: {photo_cropped_original_